<a href="https://www.kaggle.com/code/anasnofal/ai-agents-crewai?scriptVersionId=258504521" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install -qU crewai[tools,agentops]==0.114.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.5/285.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.3/573.3 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.5 MB/s et

In [2]:
!pip install -qU tavily-python scrapegraph-py

In [3]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource
import agentops
from google.colab import userdata
from pydantic import BaseModel, Field
from typing import List
from tavily import TavilyClient
from scrapegraph_py import Client

import os
import json

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ["GEMINI_API_KEY"] = user_secrets.get_secret("gemini_api_key")
os.environ["AGENTOPS_API_KEY"] = user_secrets.get_secret("agentops")

In [5]:
agentops.init(
    api_key=user_secrets.get_secret("agentops"),
    skip_auto_end_session=True,
    default_tags=['crewai']
)

In [6]:
print(agentops.get_client().config.exporter_endpoint)

https://otlp.agentops.ai/v1/traces


In [7]:
output_dir = "./ai-agent-output"
os.makedirs(output_dir, exist_ok=True)

basic_llm = LLM(model="gemini/gemini-2.0-flash", temperature=0)
search_client = TavilyClient(api_key=user_secrets.get_secret("tvly-search"))
scrape_client = Client(api_key=user_secrets.get_secret('scrapegraph'))

In [8]:
no_keywords=10
about_company = "Rankyx is a company that provides AI solutions to help websites refine their search and recommendation systems."

company_context = StringKnowledgeSource(
    content=about_company
)

## Setup Agents

### agent :A

In [9]:
class SuggestedSearchQueries(BaseModel):
    queries: List[str] = Field(..., title="Suggested search queries to be passed to the search engine",
                               min_items=1, max_items=no_keywords)

search_queries_recommendation_agent = Agent(
    role="Search Queries Recommendation Agent",
    goal="\n".join([
                "To provide a list of suggested search queries to be passed to the search engine.",
                "The queries must be varied and looking for specific items."
            ]),
    backstory="The agent is designed to help in looking for products by providing a list of suggested search queries to be passed to the search engine based on the context provided.",
    llm=basic_llm,
    verbose=True,
)

search_queries_recommendation_task = Task(
    description="\n".join([
        "Rankyx is looking to buy {product_name} at the best prices (value for a price strategy)",
        "The campany target any of these websites to buy from: {websites_list}",
        "The company wants to reach all available proucts on the internet to be compared later in another stage.",
        "The stores must sell the product in {country_name}",
        "Generate at maximum {no_keywords} queries.",
        "The search keywords must be in {language} language.",
        "Search keywords must contains specific brands, types or technologies. Avoid general keywords.",
        "The search query must reach an ecommerce webpage for product, and not a blog or listing page."
    ]),
    expected_output="A JSON object containing a list of suggested search queries with the suggested website.",
    output_json=SuggestedSearchQueries,
    output_file=os.path.join(output_dir, "step_1_suggested_search_queries.json"),
    agent=search_queries_recommendation_agent
)

### Agent:B

In [10]:
class SignleSearchResult(BaseModel):
    title: str
    url: str = Field(..., title="the page url")
    content: str
    score: float
    search_query: str

class AllSearchResults(BaseModel):
    results: List[SignleSearchResult]

@tool
def search_engine_tool(query:str):
    """Useful for search-based queries. Use this to find current information about any query related pages using a search engine"""
    return search_client.search(query)

search_engine_agent = Agent(
    role="Search Engine Agent",
    goal="To search for products based on the suggested search query",
    backstory="The agent is designed to help in looking for products by searching for products based on the suggested search queries.",
    llm=basic_llm,
    verbose=True,
    tools=[search_engine_tool]
)

search_engine_task = Task(
    description="\n".join([
        "The task is to search for products based on the suggested search queries.",
        "You have to collect results from multiple search queries.",
        "Ignore any susbicious links or not an ecommerce single product website link.",
        "Ignore any search results with confidence score less than ({score_th}) .",
        "The search results will be used to compare prices of products from different websites.",
    ]),
    expected_output="A JSON object containing the search results.",
    output_json=AllSearchResults,
    output_file=os.path.join(output_dir, "step_2_search_results.json"),
    agent=search_engine_agent
)


### Agent:C

In [11]:
class ProductSpec(BaseModel):
    specification_name: str
    specification_value: str
    
class SingleExtractedProduct(BaseModel):
    page_url: str = Field(..., title="The original url of the product page")
    product_title: str = Field(..., title="The title of the product")
    product_image_url: str = Field(..., title="The url of the product image")
    product_url: str = Field(..., title="The url of the product")
    product_current_price: float = Field(..., title="The current price of the product")
    product_original_price: float = Field(title="The original price of the product before discount. Set to None if no discount", default=None)
    product_discount_percentage: float = Field(title="The discount percentage of the product. Set to None if no discount", default=None)

    product_specs: List[ProductSpec] = Field(..., title="The specifications of the product. Focus on the most important specs to compare.", min_items=1, max_items=5)
    product_rating: float= Field(title=" the product ranking out of 5 based on the user experecnec. Set to None if it doesn't exist", default=None)
    agent_recommendation_rank: int = Field(..., title="The rank of the product to be considered in the final procurement report. (out of 5, Higher is Better) in the recommendation list ordering from the best to the worst")
    agent_recommendation_notes: List[str]  = Field(..., title="A set of notes why would you recommend or not recommend this product to the company, compared to other products.")
    
class AllExtractedProducts(BaseModel):
    products: List[SingleExtractedProduct]

@tool
def web_scraping_tool(page_url:str):
    """
    An AI Tool to help an agent to scrape a web page

    Example:
    web_scraping_tool(
        page_url="https://www.noon.com/egypt-en/15-bar-fully-automatic-espresso-machine-1-8-l-1500"
    )
    """
    details = client.smartscraper(
    website_url=page_url,
    user_prompt="Extract ```json/n" + SingleExtractedProduct.schema_json() + "```\n from the web page "
    )
    return {
        "page_url": page_url,
        "details": details
        }


scraping_agent = Agent(
    role="Web scraping agent",
    goal="To extract details from any website",
    backstory="The agent is designed to help in looking for required values from any website url. These details will be used to decide which best product to buy.",
    llm=basic_llm,
    tools=[web_scraping_tool],
    verbose=True,
)

scraping_task = Task(
    description="\n".join([
        "The task is to extract product details from any ecommerce store page url.",
        "The task has to collect results from multiple pages urls.",
        "Collect the best {top_recommendations_no} products from the search results.",
    ]),
    expected_output="A JSON object containing products details",
    output_json=AllExtractedProducts,
    output_file=os.path.join(output_dir, "step_3_search_results.json"),
    agent=scraping_agent
)

### agent d

In [12]:
procurement_report_author_agent = Agent(
    role="Procurement Report Author Agent",
    goal="To generate a professional HTML page for the procurement report",
    backstory="The agent is designed to assist in generating a professional HTML page for the procurement report after looking into a list of products.",
    llm=basic_llm,
    verbose=True,
)

procurement_report_author_task = Task(
    description="\n".join([
        "The task is to generate a professional HTML page for the procurement report.",
        "You have to use Bootstrap CSS framework for a better UI.",
        "Use the provided context about the company to make a specialized report.",
        "The report will include the search results and prices of products from different websites.",
        "The report should be structured with the following sections:",
        "1. Executive Summary: A brief overview of the procurement process and key findings.",
        "2. Introduction: An introduction to the purpose and scope of the report.",
        "3. Methodology: A description of the methods used to gather and compare prices.",
        "4. Findings: Detailed comparison of prices from different websites, including tables and charts.",
        "5. Analysis: An analysis of the findings, highlighting any significant trends or observations.",
        "6. Recommendations: Suggestions for procurement based on the analysis.",
        "7. Conclusion: A summary of the report and final thoughts.",
        "8. Appendices: Any additional information, such as raw data or supplementary materials.",
    ]),

    expected_output="A professional HTML page for the procurement report.",
    output_file=os.path.join(output_dir, "step_4_procurement_report.html"),
    agent=procurement_report_author_agent,
)

## Run the Ai Crew


In [13]:
rankyx_crew = Crew(
    agents=[
        search_queries_recommendation_agent,
    search_engine_agent,scraping_agent,procurement_report_author_agent],
    tasks=[search_queries_recommendation_task,search_engine_task,scraping_task,procurement_report_author_task
          ],
    process= Process.sequential,
    knowledge_sources=[company_context]
)

In [14]:
crew_results = rankyx_crew.kickoff(
    inputs={
        "product_name": "laptop stand for macbook m1 ",
        "websites_list": ["www.bol.com", "www.amazon.nl", "www.marktplaats.nl"],
        "country_name": "Netherlands",
        "no_keywords": 10,
        "language": "English",
        "score_th": 0.10,
        "top_recommendations_no" : 5
    }
)

15:32:59 - LiteLLM:INFO: utils.py:2896 - 
LiteLLM completion() model= gemini-2.0-flash; provider = gemini


# Agent: Search Queries Recommendation Agent
## Task: Rankyx is looking to buy laptop stand for macbook m1  at the best prices (value for a price strategy)
The campany target any of these websites to buy from: ['www.bol.com', 'www.amazon.nl', 'www.marktplaats.nl']
The company wants to reach all available proucts on the internet to be compared later in another stage.
The stores must sell the product in Netherlands
Generate at maximum 10 queries.
The search keywords must be in English language.
Search keywords must contains specific brands, types or technologies. Avoid general keywords.
The search query must reach an ecommerce webpage for product, and not a blog or listing page.


15:33:01 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler




# Agent: Search Queries Recommendation Agent
## Final Answer: 
```json
{
  "queries": [
    "ergonomic laptop stand for Macbook M1 adjustable aluminum Netherlands",
    "portable laptop stand Macbook M1 foldable lightweight Netherlands",
    "laptop stand Macbook M1 with cooling fan Netherlands bol.com",
    "Roost laptop stand alternative Macbook M1 Netherlands amazon.nl",
    "laptop stand Macbook M1 adjustable height ergonomic Netherlands marktplaats.nl",
    "aluminum laptop stand Macbook M1 space gray Netherlands",
    "laptop stand Macbook M1 with phone holder Netherlands",
    "Nexstand laptop stand Macbook M1 compatible Netherlands",
    "folding laptop stand Macbook M1 travel Netherlands",
    "laptop stand Macbook M1 vertical storage Netherlands"
  ]
}
```




15:33:03 - LiteLLM:INFO: utils.py:2896 - 
LiteLLM completion() model= gemini-2.0-flash; provider = gemini


...



# Agent: Web scraping agent
## Final Answer: 

I apologize, but due to the persistent errors with the web_scraping_tool, I am unable to fulfill the request to extract product details from the provided URLs and construct the final JSON output. The tool is not functioning as expected, preventing me from accessing and processing the necessary information.





15:33:51 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler
15:33:51 - LiteLLM:INFO: utils.py:2896 - 
LiteLLM completion() model= gemini-2.0-flash; provider = gemini


# Agent: Procurement Report Author Agent
## Task: The task is to generate a professional HTML page for the procurement report.
You have to use Bootstrap CSS framework for a better UI.
Use the provided context about the company to make a specialized report.
The report will include the search results and prices of products from different websites.
The report should be structured with the following sections:
1. Executive Summary: A brief overview of the procurement process and key findings.
2. Introduction: An introduction to the purpose and scope of the report.
3. Methodology: A description of the methods used to gather and compare prices.
4. Findings: Detailed comparison of prices from different websites, including tables and charts.
5. Analysis: An analysis of the findings, highlighting any significant trends or observations.
6. Recommendations: Suggestions for procurement based on the analysis.
7. Conclusion: A summary of the report and final thoughts.
8. Appendices: Any additional in

15:34:02 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler




# Agent: Procurement Report Author Agent
## Final Answer: 
```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Procurement Report - Laptop Stands for Macbook M1</title>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 20px;
        }
        .section-title {
            margin-top: 30px;
            margin-bottom: 20px;
            color: #333;
        }
        table {
            width: 100%;
            border-collapse: collapse;
            margin-bottom: 20px;
        }
        th, td {
            border: 1px solid #ddd;
            padding: 8px;
            text-align: left;
        }
        th {
            background-color: #f2f2f2;
        }
        .executive-summary {
            background-color: #f8f9fa;
          

In [15]:
crew_results

CrewOutput(raw='```html\n<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>Procurement Report - Laptop Stands for Macbook M1</title>\n    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">\n    <style>\n        body {\n            font-family: Arial, sans-serif;\n            margin: 20px;\n        }\n        .section-title {\n            margin-top: 30px;\n            margin-bottom: 20px;\n            color: #333;\n        }\n        table {\n            width: 100%;\n            border-collapse: collapse;\n            margin-bottom: 20px;\n        }\n        th, td {\n            border: 1px solid #ddd;\n            padding: 8px;\n            text-align: left;\n        }\n        th {\n            background-color: #f2f2f2;\n        }\n        .executive-summary {\n            background-color: #f8f9fa;\n            padding: 1